In [1]:
import warnings
warnings.simplefilter("ignore", UserWarning)

from __future__ import print_function
from pytorch_lightning.metrics.functional import accuracy

from PIL import Image

import pickle
import os
import random
import numpy as np
import shutil
import copy

import numpy as np
import pandas as pd

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


import torch
import pytorch_lightning as pl
import os

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

from torch.nn import functional as F
import torchmetrics

from pytorch_lightning.loggers import TensorBoardLogger
from pl_bolts.callbacks import PrintTableMetricsCallback


import math
import torchvision.models as models
from PIL import Image
import cv2
import time

from skimage import io as iio
from pytorch_lightning.trainer.supporters import CombinedLoader

In [2]:
data_transform = transforms.Compose([
                            transforms.Resize(size = (224, 224)),
                            transforms.Grayscale(num_output_channels=3),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [3]:
class pre_resnest50(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size



        backbone = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        
        self.feature_extractor = nn.Sequential(*layers)

        num_target_classes = 800
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)


        return F.log_softmax(x, dim=1)

class pre_resnest50_wo_freeze(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50_wo_freeze, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()

        resnest50 = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        trained_resnest50 = pre_resnest50.load_from_checkpoint('pre_resnest50_epoch=54-val_loss=6.26.ckpt', learning_rate=0.08, batch_size=128)

        layers_1 = list(resnest50.children())[:-1][0]
        layers_2 = list(resnest50.children())[:-1][1:-2]
        layers_3 = list(resnest50.children())[:-1][-2:]
        self.classifier = list(trained_resnest50.children())[-1]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)



In [4]:

class t_brain_right(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(t_brain_right, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()


        tbrain = pre_resnest50_wo_freeze.load_from_checkpoint('resnest50_tbrain_epoch=49-val_loss=0.056.ckpt')

        layers_1 = list(tbrain.children())[1]
        layers_2 = list(tbrain.children())[2]
        layers_3 = list(tbrain.children())[3]
        self.classifier = list(tbrain.children())[0]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)


class t_brain_energy(pl.LightningModule):
    def __init__(self, learning_rate, batch_size, m_in ,m_out, temperature):
        
        super(t_brain_energy, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.m_in = m_in
        self.m_out = m_out
        self.temperature = temperature
        self.save_hyperparameters()


        tbrain = t_brain_right.load_from_checkpoint('resnest50_tbrain_right_epoch=47-val_loss=0.047.ckpt')

        layers_1 = list(tbrain.children())[1]
        layers_2 = list(tbrain.children())[2]
        layers_3 = list(tbrain.children())[3]
        self.classifier = list(tbrain.children())[0]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    

        return self.classifier(x)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classfication_model = t_brain_right.load_from_checkpoint('resnest50_tbrain_right_epoch=47-val_loss=0.047.ckpt')
ood_model = t_brain_energy.load_from_checkpoint('t_brain_energy_epoch=22-val_loss=0.120.ckpt')

classfication_model.to(device)
ood_model.to(device)

classfication_model.eval()
ood_model.eval()

Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master


t_brain_energy(
  (classifier): Linear(in_features=2048, out_features=800, bias=True)
  (seq_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (seq_2): Sequential(
    (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, mome

In [6]:
a_file = open("index_to_char.pkl", "rb")
index_to_char = pickle.load(a_file)

In [35]:
t1 = time.time()
img_abs = 'C:/Users/bbb50/Desktop/Python/OCR/test_data\\1129.jpg' ## 讀入的照片
img_g = cv2.imread(img_abs, cv2.IMREAD_GRAYSCALE)
img_g = cv2.resize(img_g, (224,224), interpolation=cv2.INTER_CUBIC)
_, img_p = cv2.threshold(img_g,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
img_3c = Image.fromarray(cv2.cvtColor(img_p, cv2.COLOR_GRAY2BGR),mode="RGB")

img_input = data_transform(img_3c)
img_input = img_input.reshape(1,*img_input.shape).to(device)

logits = F.log_softmax(classfication_model(img_input), dim=1)
out = ood_model(img_input)
score = -(1*torch.logsumexp(out / 1, dim=1)).item()

if score > 5:
    char_return = "isnull"
else:
    char_return = index_to_char[torch.argmax(logits, dim=1)[0].item()]
t2 = time.time()
print('time:', t2 - t1)

time: 0.10458183288574219


In [52]:


t2 = time.time()
print('time:', t2 - t1)

time: 0.5136237144470215


In [25]:
print(char_return) #回傳的字型
print('time:', t2 - t1)

宋
time: 1.0375275611877441
